In [1]:
import os
import tarfile
import urllib
import time
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import seaborn as sb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer

In [2]:
from google.colab import files
from google.colab import drive

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"benjaminbutch","key":"db2f86f61ac7a18276861c2a7721213b"}'}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [6]:
!kaggle datasets download -d dansbecker/melbourne-housing-snapshot

  0% 0.00/451k [00:00<?, ?B/s]
100% 451k/451k [00:00<00:00, 64.5MB/s]


In [7]:
!rm -fr /content/Melbourne_housing/

In [8]:
!unzip -q melbourne-housing-snapshot.zip

In [9]:
def load_housing_data(housing_path = "/content/"):
  csv_path = os.path.join(housing_path,'melb_data.csv')
  return pd.read_csv(csv_path)

Raw_housing_data = load_housing_data()

Rooms: Number of rooms

Price: Price in dollars

Method: S - property sold; SP - property sold prior; PI - property passed in; PN - sold prior not disclosed; SN - sold not disclosed; NB - no bid; VB - vendor bid; W - withdrawn prior to auction; SA - sold after auction; SS - sold after auction price not disclosed. N/A - price or highest bid not available.

Type: br - bedroom(s); h - house,cottage,villa, semi,terrace; u - unit, duplex; t - townhouse; dev site - development site; o res - other residential.

SellerG: Real Estate Agent

Date: Date sold

Distance: Distance from CBD

Regionname: General Region (West, North West, North, North east …etc)

Propertycount: Number of properties that exist in the suburb.

Bedroom2 : Scraped # of Bedrooms (from different source)

Bathroom: Number of Bathrooms

Car: Number of carspots

Landsize: Land Size

BuildingArea: Building Size

CouncilArea: Governing council for the area

In [10]:
Raw_housing_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


What we can see from Rooms is that the number of rooms doesnt intake the number of bedrooms or bathrooms as we see from row 0 that there are 2 bedrooms and 1 bathroom but listed as only 2 rooms meanting there are actually 5 rooms to fix this we do:

In [11]:
Raw_housing_data["Rooms"] = Raw_housing_data["Rooms"] + Raw_housing_data["Bedroom2"] + Raw_housing_data["Bathroom"]

In [12]:
Raw_housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  float64
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

Here we can see we are missing data in Car spaces, Building Area, Year Built and Council area

We can take the Null values of Car and Council area and set them to 0 and false as if there isnt any data on them it is likely that there arent any in the first place or isnt in the area

for building area and year build we are going to have to use a pipeline to fix this using the median method.

In [13]:
Raw_housing_data.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,7.386966,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,2.373503,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,2.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,5.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,7.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,9.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,24.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [14]:
Raw_housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  float64
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [15]:
from sklearn.model_selection import StratifiedShuffleSplit
#this code is for sorting our dataset to be more understandable
Raw_housing_data['Bedrooms'] = Raw_housing_data['Bedroom2']
Raw_housing_data =Raw_housing_data.drop('Bedroom2',axis=1)
Raw_housing_data['BuildingSize'] = Raw_housing_data['BuildingArea']
Raw_housing_data =Raw_housing_data.drop('BuildingArea',axis=1)
Raw_housing_data['NO_CarSpots'] = Raw_housing_data['Car']
Raw_housing_data =Raw_housing_data.drop('Car',axis=1)
Raw_housing_data['Latitude'] = Raw_housing_data['Lattitude']
Raw_housing_data =Raw_housing_data.drop('Lattitude',axis=1)
Raw_housing_data['Longitude'] = Raw_housing_data['Longtitude']
Raw_housing_data =Raw_housing_data.drop('Longtitude',axis=1)
Raw_housing_data['Price_cat'] = pd.cut(Raw_housing_data['Price'],
                                bins = [0.0,900000,3000000,6000000,9000000,np.inf],
                                labels = [1,2,3,4,5])
#splits Price into 5 categories from low to high






In [16]:
Raw_housing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Suburb         13580 non-null  object  
 1   Address        13580 non-null  object  
 2   Rooms          13580 non-null  float64 
 3   Type           13580 non-null  object  
 4   Price          13580 non-null  float64 
 5   Method         13580 non-null  object  
 6   SellerG        13580 non-null  object  
 7   Date           13580 non-null  object  
 8   Distance       13580 non-null  float64 
 9   Postcode       13580 non-null  float64 
 10  Bathroom       13580 non-null  float64 
 11  Landsize       13580 non-null  float64 
 12  YearBuilt      8205 non-null   float64 
 13  CouncilArea    12211 non-null  object  
 14  Regionname     13580 non-null  object  
 15  Propertycount  13580 non-null  float64 
 16  Bedrooms       13580 non-null  float64 
 17  BuildingSize   7130 non-null   

In [17]:

for i in range(0,len(Raw_housing_data['Date'])):
  X = Raw_housing_data['Date']
  if X[i] != 0 :
                   t = time.strptime(X[i], '%d/%m/%Y')
                   X[i] = time.strftime('%d%m%Y', t)
  else:
    X[i] =3102015
Raw_housing_data['Date'] = X


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
Raw_housing_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bathroom,Landsize,YearBuilt,CouncilArea,Regionname,Propertycount,Bedrooms,BuildingSize,NO_CarSpots,Latitude,Longitude,Price_cat
0,Abbotsford,85 Turner St,5.0,h,1480000.0,S,Biggin,03122016,2.5,3067.0,1.0,202.0,NaN,Yarra,Northern Metropolitan,4019.0,2.0,NaN,1.0,-37.7996,144.9984,2
1,Abbotsford,25 Bloomburg St,5.0,h,1035000.0,S,Biggin,04022016,2.5,3067.0,1.0,156.0,1900.0,Yarra,Northern Metropolitan,4019.0,2.0,79.0,0.0,-37.8079,144.9934,2
2,Abbotsford,5 Charles St,8.0,h,1465000.0,SP,Biggin,04032017,2.5,3067.0,2.0,134.0,1900.0,Yarra,Northern Metropolitan,4019.0,3.0,150.0,0.0,-37.8093,144.9944,2
3,Abbotsford,40 Federation La,8.0,h,850000.0,PI,Biggin,04032017,2.5,3067.0,2.0,94.0,NaN,Yarra,Northern Metropolitan,4019.0,3.0,NaN,1.0,-37.7969,144.9969,1
4,Abbotsford,55a Park St,8.0,h,1600000.0,VB,Nelson,04062016,2.5,3067.0,1.0,120.0,2014.0,Yarra,Northern Metropolitan,4019.0,3.0,142.0,2.0,-37.8072,144.9941,2


In [19]:
rooms_ix,bedrooms_ix,landsize_ix,carspots_ix = 2,16,11,18

class CombinedAttributesAdder(BaseEstimator,TransformerMixin):
  def __init__(self,add_bedrooms_per_room = True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self,X,y=None):
    return self
  def transform(self,X,y=None):
    parking_per_landsize = np.empty(13580, dtype=object)
    rooms_per_landsize = np.empty(13580, dtype=object)
    for i in range(0,(len(X[:,landsize_ix]))):
      print(i)
      if X[i,carspots_ix] != 0 and X[i,landsize_ix] != 0 :
        
        
        
        parking_per_landsize[i] = X[i,carspots_ix] / X[i,landsize_ix]
      else:
        parking_per_landsize[i] = 0
    for i in range(0,(len(X[:,rooms_ix]))):
      if X[i,rooms_ix] != 0 and X[i,landsize_ix] != 0:
        rooms_per_landsize[i] = X[i,rooms_ix] / X[i,landsize_ix]
      else:
        rooms_per_landsize[i] = 3
        
    if self.add_bedrooms_per_room:
      bedrooms_per_room = X[:,bedrooms_ix] / X[:,rooms_ix]
      return np.c_[X,rooms_per_landsize,parking_per_landsize,bedrooms_per_room]
    else:
      return np.c_[X,rooms_per_landsize,parking_per_landsize]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room = True)
housing_extra_attribs = attr_adder.transform(Raw_housing_data.values)
#Class to combine attributes with parameter that lets us torn on/off bedrooms per room

In [20]:
Raw_housing_data["Bedrooms_per_Room"] = Raw_housing_data["Bedrooms"] / Raw_housing_data["Rooms"]
Raw_housing_data["Rooms_per_Landsize"] = Raw_housing_data["Rooms"]/Raw_housing_data["Landsize"]
Raw_housing_data["NO_CarSpots_per_Landsize"] = Raw_housing_data["NO_CarSpots"]/Raw_housing_data["Landsize"]

In [21]:
#sort Non-float values into float values

In [34]:
Coucil_Area = Raw_housing_data['CouncilArea']
House_Type = Raw_housing_data['Type']
House_Address = Raw_housing_data['Address']
House_Suburb = Raw_housing_data['Suburb']
House_Seller = Raw_housing_data['SellerG']
House_Region = Raw_housing_data['Regionname']
Seller_Method = Raw_housing_data['Method']

housing_num = Raw_housing_data.drop("CouncilArea",axis=1)
housing_num = housing_num.drop("Type",axis=1)
housing_num = housing_num.drop("Address",axis=1)
housing_num = housing_num.drop("Suburb",axis=1)
housing_num = housing_num.drop("SellerG",axis=1)
housing_num = housing_num.drop("Regionname",axis=1)
housing_num = housing_num.drop("Method",axis=1)
housing_num.head()

,Rooms,Price,Date,Distance,Postcode,Bathroom,Landsize,YearBuilt,Propertycount,Bedrooms,BuildingSize,NO_CarSpots,Latitude,Longitude,Price_cat,Bedrooms_per_Room,Rooms_per_Landsize,NO_CarSpots_per_Landsize
0,5.0,1480000.0,03122016,2.5,3067.0,1.0,202.0,NaN,4019.0,2.0,NaN,1.0,-37.7996,144.9984,2,0.400,0.024752,0.004950
1,5.0,1035000.0,04022016,2.5,3067.0,1.0,156.0,1900.0,4019.0,2.0,79.0,0.0,-37.8079,144.9934,2,0.400,0.032051,0.000000
2,8.0,1465000.0,04032017,2.5,3067.0,2.0,134.0,1900.0,4019.0,3.0,150.0,0.0,-37.8093,144.9944,2,0.375,0.059701,0.000000
3,8.0,850000.0,04032017,2.5,3067.0,2.0,94.0,NaN,4019.0,3.0,NaN,1.0,-37.7969,144.9969,1,0.375,0.085106,0.010638
4,8.0,1600000.0,04062016,2.5,3067.0,1.0,120.0,2014.0,4019.0,3.0,142.0,2.0,-37.8072,144.9941,2,0.375,0.066667,0.016667


In [35]:
housing_num.replace([np.inf, -np.inf], np.nan, inplace=True)
housing_num.head()


,Rooms,Price,Date,Distance,Postcode,Bathroom,Landsize,YearBuilt,Propertycount,Bedrooms,BuildingSize,NO_CarSpots,Latitude,Longitude,Price_cat,Bedrooms_per_Room,Rooms_per_Landsize,NO_CarSpots_per_Landsize
0,5.0,1480000.0,03122016,2.5,3067.0,1.0,202.0,NaN,4019.0,2.0,NaN,1.0,-37.7996,144.9984,2,0.400,0.024752,0.004950
1,5.0,1035000.0,04022016,2.5,3067.0,1.0,156.0,1900.0,4019.0,2.0,79.0,0.0,-37.8079,144.9934,2,0.400,0.032051,0.000000
2,8.0,1465000.0,04032017,2.5,3067.0,2.0,134.0,1900.0,4019.0,3.0,150.0,0.0,-37.8093,144.9944,2,0.375,0.059701,0.000000
3,8.0,850000.0,04032017,2.5,3067.0,2.0,94.0,NaN,4019.0,3.0,NaN,1.0,-37.7969,144.9969,1,0.375,0.085106,0.010638
4,8.0,1600000.0,04062016,2.5,3067.0,1.0,120.0,2014.0,4019.0,3.0,142.0,2.0,-37.8072,144.9941,2,0.375,0.066667,0.016667


In [37]:

num_pipeline = Pipeline([('imputer',SimpleImputer(strategy="median")),
                          ('attribs_adder',CombinedAttributesAdder()),
                          ('std_scaler',StandardScaler()),
                          ])
housing_num_tr = num_pipeline.fit_transform(housing_num)

num_attribs = list(housing_num)
cat_attribs = ["CouncilArea","Regionname","Type","Suburb","Address","SellerG","Method"]

full_pipeline = ColumnTransformer([("num",num_pipeline,num_attribs),
                                   ("cat",OneHotEncoder(),cat_attribs,)])

Raw_housing_data = full_pipeline.fit_transform(Raw_housing_data)
#Fill missing data


IndexError: ignored

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(Raw_housing_data,Raw_housing_data['Price_cat']):
  Housing_train_set = Raw_housing_data.loc[train_index]
  Housing_test_set = Raw_housing_data.loc[test_index]
  
#creates a stratified training set and test set to remove most bais we can put into the AI

In [ ]:
Raw_housing_data.head()

In [ ]:
Housing_train_set.plot(kind='scatter',x='Longtitude',y='Lattitude',alpha = 0.4)

This Scatter graph shows the data in the shape of the map of melbourne, we can see most properties in the main city of melbourne and along the coast 

In [ ]:
Housing_train_set.plot(kind='scatter',x='Longtitude',y='Lattitude',alpha = 0.2,s=Housing_train_set['Propertycount']/500,label='Property count',figsize=(20,14),c='Price',cmap=plt.get_cmap('jet'),colorbar=True)

plt.legend()

From this graph we can see that the propertys on the higher end of the spectrum are located near the coast and in the middle of melbourne whereas the ones on the lower end are near the harbour and industrial areas

In [ ]:
mask = np.zeros_like(Housing_train_set.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sb.heatmap(Housing_train_set.corr()*100,cmap='RdBu_r',annot= True, mask=mask)

From this heatmap we can see what we would expect e.g. more bathrooms = more bedrooms, year built means less expensive, more rooms = higher price etc etc.